# Anomaly Detection with Isolation Forest - Manual Exploration

This notebook demonstrates anomaly detection with Isolation Forest for static code analysis gathered by using jQAssistant and Neo4j. Detecting anomalies in the data can be useful for identifying potential issues or areas for improvement in the codebase. To explain the results, we use SHAP (SHapley Additive exPlanations) to provide insights into the feature importances and how they contribute to the anomaly scores.

<br>  

### References
- [jqassistant](https://jqassistant.org)
- [Neo4j Python Driver](https://neo4j.com/docs/api/python-driver/current)

## Features overview

| **Feature**                      | **Type**           | **What it Measures**                        | **Why It’s Useful**                         |
| -------------------------------- | ------------------ | ------------------------------------------- | ------------------------------------------- |
| `PageRank`                       | Centrality         | Popularity / referenced code                | High = many dependents                      |
| `ArticleRank`                    | Centrality         | How much the code depends on others         | High = high dependency                      |
| `PageRank - ArticleRank`         | Relative Rank      | Role inversion / architectural layering     | Highlights mismatches                       |
| `Betweenness Centrality`         | Centrality         | Bridge or control nodes                     | High = structural chokepoints               |
| `Local Clustering Coefficient`   | Structural         | Local cohesion / modularity                 | Low = isolated node in a clique-like region |
| `Degree` (Total and In)         | Structural         | Connectivity                                | Raw values may dominate                     |
| `Node Embedding` (PCA reduced)   | Latent             | Structural and semantic similarity          | Captures latent position in graph           |
| `Normalized Cluster Distance`    | Geometric          | Relative to cluster radius                  | Adds context to position                    |
| `1.0 - HDBSCAN membership probability` | Cluster Confidence | How confidently HDBSCAN clustered this node, 1-x inverted | High score = likely anomaly                   |
| `Average Cluster Radius`          | Cluster Context    | How tight or spread out the cluster is         | Highly spread clusters may be a less meaningful one   |



In [ ]:
import typing
import numpy.typing as numpy_typing

import os
from IPython.display import display

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.model_selection import cross_val_score

from optuna.importance import get_param_importances, MeanDecreaseImpurityImportanceEvaluator
from optuna.trial import TrialState
from optuna.samplers import TPESampler
from optuna import Study, create_study

import shap # Explainable AI tool

import matplotlib.pyplot as plot

In [ ]:
#The following cell uses the build-in %html "magic" to override the CSS style for tables to a much smaller size.
#This is especially needed for PDF export of tables with multiple columns.

In [ ]:
%%html
<style>
/* CSS style for smaller dataframe tables. */
.dataframe th {
    font-size: 8px;
}
.dataframe td {
    font-size: 8px;
}
</style>

In [ ]:
# Main Colormap
# main_color_map = 'nipy_spectral'
main_color_map = 'viridis'

In [ ]:
from sys import version as python_version
print('Python version: {}'.format(python_version))

from numpy import __version__ as numpy_version
print('numpy version: {}'.format(numpy_version))

from pandas import __version__ as pandas_version
print('pandas version: {}'.format(pandas_version))

from sklearn import __version__ as sklearn_version
print('sklearn version: {}'.format(sklearn_version))

from matplotlib import __version__ as matplotlib_version
print('matplotlib version: {}'.format(matplotlib_version))

from neo4j import __version__ as neo4j_version
print('neo4j version: {}'.format(neo4j_version))

from optuna import __version__ as optuna_version
print('optuna version: {}'.format(optuna_version))

In [ ]:
# Please set the environment variable "NEO4J_INITIAL_PASSWORD" in your shell 
# before starting jupyter notebook to provide the password for the user "neo4j". 
# It is not recommended to hardcode the password into jupyter notebook for security reasons.
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    uri="bolt://localhost:7687", 
    auth=("neo4j", os.environ.get("NEO4J_INITIAL_PASSWORD"))
)
driver.verify_connectivity()

In [ ]:
def query_cypher_to_data_frame(query: typing.LiteralString, parameters: typing.Optional[typing.Dict[str, typing.Any]] = None):
    records, summary, keys = driver.execute_query(query, parameters_=parameters)
    return pd.DataFrame([record.values() for record in records], columns=keys)

In [ ]:
plot_annotation_style: dict = {
    'textcoords': 'offset points',
    'arrowprops': dict(arrowstyle='->', color='black', alpha=0.3),
    'fontsize': 6,
    'backgroundcolor': 'white',
    'bbox': dict(boxstyle='round,pad=0.4',
                    edgecolor='silver',
                    facecolor='whitesmoke',
                    alpha=1
                )
}

In [ ]:
# Global Settings

# -> Feature "outgoingDependencies" is left in even though it is highly correlated with the feature "degree" and isn't important for anomaly decisions according to SHAP. Still, the F1 score of the model is improved by it. 
# -> Feature "articleRank" is left in even though it is highly correlated with the feature "incomingDependencies". It is important for anomaly decisions according to SHAP and improves the F1 score. 
features_for_visualization_excluded_from_training: typing.List[str] = [
    'codeUnitName',
    'shortCodeUnitName',
    'projectName',
    'clusterLabel',
    'clusterSize',
    'clusterMedoid',
    'clusterNoise', # highly correlated with "clusterApproximateOutlierScore". doesn't improve F1 score of proxy model.
    'embeddingVisualizationX',
    'embeddingVisualizationY',
]

## 1. Java Packages

### 1.1 Query Features

Query all features that are relevant for anomaly detection. Some of them come from precalculated clustering (HDBSCAN), node embeddings (Fast Random Projection), community detection algorithms (Leiden, Local Clustering Coefficient), centrality algorithms (Page Rank, Article Rank, Betweenness) and classical metrics like the in-/out-degree.

In [ ]:
java_package_anomaly_detection_features_query = """
    MATCH (artifact:Java:Artifact)-[:CONTAINS]->(codeUnit:Java:Package)
    WHERE codeUnit.incomingDependencies                              IS NOT NULL
      AND codeUnit.outgoingDependencies                              IS NOT NULL
      and codeUnit.embeddingsFastRandomProjectionTunedForClustering  IS NOT NULL
      AND codeUnit.centralityPageRank                                IS NOT NULL
      AND codeUnit.centralityArticleRank                             IS NOT NULL
      AND codeUnit.centralityBetweenness                             IS NOT NULL
      AND codeUnit.communityLocalClusteringCoefficient               IS NOT NULL
      AND codeUnit.clusteringHDBSCANProbability                      IS NOT NULL
      AND codeUnit.clusteringHDBSCANNoise                            IS NOT NULL
      AND codeUnit.clusteringHDBSCANMedoid                           IS NOT NULL
      AND codeUnit.clusteringHDBSCANRadiusAverage                    IS NOT NULL
      AND codeUnit.clusteringHDBSCANNormalizedDistanceToMedoid       IS NOT NULL
      AND codeUnit.clusteringHDBSCANSize                             IS NOT NULL
      AND codeUnit.clusteringHDBSCANLabel                            IS NOT NULL
      AND codeUnit.clusteringHDBSCANMedoid                           IS NOT NULL
      AND codeUnit.embeddingFastRandomProjectionVisualizationX       IS NOT NULL
      AND codeUnit.embeddingFastRandomProjectionVisualizationY       IS NOT NULL
   RETURN DISTINCT 
         codeUnit.fqn                                                  AS codeUnitName
        ,codeUnit.name                                                 AS shortCodeUnitName
        ,artifact.name                                                 AS projectName
        ,codeUnit.incomingDependencies                                 AS incomingDependencies
        ,codeUnit.outgoingDependencies                                 AS outgoingDependencies
        ,codeUnit.incomingDependencies + codeUnit.outgoingDependencies AS degree
        ,codeUnit.embeddingsFastRandomProjectionTunedForClustering     AS embedding
        ,codeUnit.centralityPageRank                                   AS pageRank
        ,codeUnit.centralityArticleRank                                AS articleRank
        ,codeUnit.centralityPageRank - codeUnit.centralityArticleRank  AS pageToArticleRankDifference
        ,codeUnit.centralityBetweenness                                AS betweenness
        ,codeUnit.communityLocalClusteringCoefficient                  AS locallusteringCoefficient
        ,1.0 - codeUnit.clusteringHDBSCANProbability                   AS clusterApproximateOutlierScore
        ,codeUnit.clusteringHDBSCANNoise                               AS clusterNoise
        ,codeUnit.clusteringHDBSCANRadiusAverage                       AS clusterRadiusAverage
        ,codeUnit.clusteringHDBSCANNormalizedDistanceToMedoid          AS clusterDistanceToMedoid
        ,codeUnit.clusteringHDBSCANSize                                AS clusterSize
        ,codeUnit.clusteringHDBSCANLabel                               AS clusterLabel
        ,codeUnit.clusteringHDBSCANMedoid                              AS clusterMedoid
        ,codeUnit.embeddingFastRandomProjectionVisualizationX          AS embeddingVisualizationX
        ,codeUnit.embeddingFastRandomProjectionVisualizationY          AS embeddingVisualizationY
"""

java_package_anomaly_detection_features = query_cypher_to_data_frame(java_package_anomaly_detection_features_query)
java_package_features_to_standardize = java_package_anomaly_detection_features.columns.drop(features_for_visualization_excluded_from_training + ['embedding']).to_list()

display(java_package_anomaly_detection_features.head(5))

### 1.2 Data preparation

Prepare the data by standardizing numeric fields and reducing the dimensionality of the node embeddings to not dominate the results.

In [ ]:
def validate_data(features: pd.DataFrame) -> None:
    if features.empty:
        print("Data Validation Info: No data")

    if features.isnull().values.any():
        raise RuntimeError("Data Validation Error: Some values are null. Fix the wrong values or filter them out.")

In [ ]:
validate_data(java_package_anomaly_detection_features)

In [ ]:
# Check for correlation between features for debugging, troubleshooting, and detailed analysis.

def plot_feature_correlation_matrix(features: pd.DataFrame) -> None:
    """
    Plots the correlation matrix of the features in the DataFrame.
    
    :param java_package_anomaly_detection_features: DataFrame containing the features.
    :param java_package_features_to_standardize: List of feature names to include in the correlation matrix.
    """
    correlation_matrix = features.corr()

    figure, axis = plot.subplots(figsize=(8, 6))
    color_axis = axis.matshow(correlation_matrix, cmap="coolwarm")
    figure.colorbar(color_axis)
    axis.set_xticks(range(len(correlation_matrix.columns)))
    axis.set_yticks(range(len(correlation_matrix.index)))
    axis.set_xticklabels(correlation_matrix.columns, rotation=90)
    axis.set_yticklabels(correlation_matrix.index)
    for (i, j), correlation_value in np.ndenumerate(correlation_matrix.values):
        axis.text(j, i, f"{correlation_value:.2f}", ha='center', va='center', color='black', bbox=dict(facecolor='white', alpha=0.3, edgecolor='none'))
    plot.title("Feature Correlation Matrix (excluding embeddings)", fontsize=10)
    plot.tight_layout()
    plot.show()

In [ ]:
plot_feature_correlation_matrix(java_package_anomaly_detection_features[java_package_features_to_standardize])

In [ ]:
def standardize_features(features: pd.DataFrame, feature_list: list[str]) -> numpy_typing.NDArray:
    features_to_scale = features[feature_list]
    scaler = StandardScaler()
    return scaler.fit_transform(features_to_scale)

In [ ]:
java_package_anomaly_detection_features_standardized = standardize_features(java_package_anomaly_detection_features, java_package_features_to_standardize)

In [ ]:
def reduce_dimensionality_of_node_embeddings(
        features: pd.DataFrame, 
        min_dimensions: int = 20, 
        max_dimensions: int = 40, 
        target_variance: float = 0.90,
        embedding_column_name: str = 'embedding'
) -> numpy_typing.NDArray:
    """
    Automatically reduce the dimensionality of node embeddings using Principal Component Analysis (PCA)
    to reach a target explained variance ratio with the lowest possible number of components (output dimensions).

    Parameters:
    - features (pd.DataFrame) with a column 'embedding', where every value contains a float array with original dimensions.
    - min_dimensions: Even if possible with the given variance, don't go below this number of dimensions for the output
    - max_dimensions: Return at most the max number of dimensions, even if that means, that the target variance can't be met.
    - target_variance (float): Cumulative variance threshold (default: 0.90)
    - embedding_column_name (string): Defaults to 'embedding'

    Returns: Reduced embeddings as an numpy array
    """

    # Convert the input and get the original dimension
    embeddings = np.stack(features[embedding_column_name].apply(np.array).tolist())
    original_dimension = embeddings.shape[1]

    # Fit PCA without dimensionality reduction to get explained variance
    full_principal_component_analysis_without_reduction = PCA()
    full_principal_component_analysis_without_reduction.fit(embeddings)

    # Find smallest number of components to reach target variance
    cumulative_variance = np.cumsum(full_principal_component_analysis_without_reduction.explained_variance_ratio_)
    best_n_components = np.searchsorted(cumulative_variance, target_variance) + 1
    best_n_components = max(best_n_components, min_dimensions) # Use at least min_dimensions
    best_n_components = min(best_n_components, max_dimensions) # Use at most max_dimensions

    # Apply PCA with optimal number of components
    principal_component_analysis = PCA(n_components=best_n_components)
    java_type_anomaly_detection_node_embeddings_reduced = principal_component_analysis.fit_transform(embeddings)

    explained_variance_ratio_sum = sum(principal_component_analysis.explained_variance_ratio_)
    print(f"Dimensionality reduction from {original_dimension} to {best_n_components} (min {min_dimensions}, max {max_dimensions}) of node embeddings using Principal Component Analysis (PCA): Explained variance is {explained_variance_ratio_sum:.4f}.")

    return java_type_anomaly_detection_node_embeddings_reduced
    

In [ ]:
java_package_anomaly_detection_node_embeddings_reduced = reduce_dimensionality_of_node_embeddings(java_package_anomaly_detection_features)

In [ ]:
java_package_anomaly_detection_features_prepared = np.hstack([java_package_anomaly_detection_features_standardized, java_package_anomaly_detection_node_embeddings_reduced])
java_package_anomaly_detection_feature_names = list(java_package_features_to_standardize) + [f'pca_{i}' for i in range(java_package_anomaly_detection_node_embeddings_reduced.shape[1])]

### 1.3 List the top 10 anomalies found using Isolation Forest

> The IsolationForest 'isolates' observations by randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature.

In [ ]:
def output_optuna_tuning_results(optimized_study: Study, name_of_the_optimized_algorithm: str):

    print(f"Best {name_of_the_optimized_algorithm} parameters (Optuna):", optimized_study.best_params)
    print(f"Best {name_of_the_optimized_algorithm} score:", optimized_study.best_value)
    print(f"Best {name_of_the_optimized_algorithm} parameter influence:", get_param_importances(optimized_study, evaluator=MeanDecreaseImpurityImportanceEvaluator()))
    
    valid_trials = [trial for trial in optimized_study.trials if trial.value is not None and trial.state == TrialState.COMPLETE]
    top_trials = sorted(valid_trials, key=lambda t: typing.cast(float, t.value), reverse=True)[:10]
    for i, trial in enumerate(top_trials):
        print(f"Best {name_of_the_optimized_algorithm} parameter rank: {i+1}, trial: {trial.number}, Value = {trial.value:.6f}, Params: {trial.params}")


class AnomalyDetectionResults:
    def __init__(self, 
                 anomaly_labels: np.ndarray, 
                 anomaly_scores: np.ndarray, 
                 random_forest_classifier: RandomForestClassifier,
                 feature_importances: np.ndarray
                 ):
        self.anomaly_labels = anomaly_labels
        self.anomaly_scores = anomaly_scores
        self.random_forest_classifier = random_forest_classifier
        self.feature_importances = feature_importances
        
    def __repr__(self):
        return (f"AnomalyDetectionResults(anomaly_labels={self.anomaly_labels.shape}, "
                f"anomaly_scores={self.anomaly_scores.shape}, "
                f"random_forest_classifier={type(self.random_forest_classifier).__name__}, "
                f"feature_importances={self.feature_importances.shape})")


def tune_anomaly_detection_models(
    feature_matrix: np.ndarray,
    contamination: float | typing.Literal["auto"] = 0.05,
    random_seed: int = 42,
    number_of_trials: int = 20,
    optimization_timeout_in_seconds: int = 40,
    number_of_cross_validation_folds: int = 3,
) -> AnomalyDetectionResults:
    """
    Tunes both Isolation Forest and a proxy Random Forest using Optuna, maximizing the F1 score
    between Isolation Forest pseudo-labels and proxy predictions. The proxy model mimics the behavior of the Isolation Forest 
    and is mainly used to provide feature importances and explainability using SHAP values later.

    Parameters:
    - feature_matrix: np.ndarray of shape (n_samples, n_features), preprocessed input features.
    - contamination: favor only a few suspicious cases with a fixed percentage
    - random_seed: seed for reproducibility.
    - number_of_trials: number of Optuna optimization trials.
    - number_of_cross_validation_folds: number of cross validation (CV) folds for proxy model validation.

    Returns:
    - AnomalyDetectionResults containing:
        - anomaly_labels: np.ndarray of shape (n_samples,), binary labels indicating anomalies (1) or normal (0).
        - anomaly_scores: np.ndarray of shape (n_samples,), anomaly scores where higher values indicate more anomalous instances.
        - random_forest_classifier: trained Random Forest classifier that mimics the Isolation Forest behavior.
        - feature_importances: np.ndarray of shape (n_features,), feature importances from the Random Forest classifier.
    """

    def objective(trial) -> float:
        # Isolation Forest hyperparameters
        isolation_forest = IsolationForest(
            max_samples=trial.suggest_float("isolation_max_samples", 0.1, 1.0),
            contamination=contamination, 
            n_estimators=trial.suggest_int("isolation_n_estimators", 100, 400),
            random_state=random_seed
        )
        # Train Isolation Forest
        isolation_forest.fit(feature_matrix)

        # Generate pseudo-labels: 1 = anomaly, 0 = normal
        pseudo_labels = (isolation_forest.predict(feature_matrix) == -1).astype(int)

        # Proxy Random Forest hyperparameters
        proxy_random_forest = RandomForestClassifier(
            n_estimators=trial.suggest_int("proxy_n_estimators", 50, 300),
            max_depth=trial.suggest_int("proxy_max_depth", 3, 20),
            random_state=random_seed,
        )

        # Train proxy model
        # Use cross-validation to get robust F1 score
        f1_scores = cross_val_score(
            proxy_random_forest,
            feature_matrix,
            pseudo_labels,
            cv=number_of_cross_validation_folds,
            scoring="f1",
        )
        return float(np.mean(f1_scores))


    # Print the number of samples and features in the feature matrix
    n_samples = feature_matrix.shape[0]
    print(f"Tuned Anomaly Detection: Number of samples: {n_samples}, Number of features: {feature_matrix.shape[1]}, Number of trials: {number_of_trials}")

    # Run Optuna optimization
    study = create_study(direction="maximize", sampler=TPESampler(seed=random_seed), study_name="AnomalyDetection_Tuning")

    # Try (enqueue) default settings
    study.enqueue_trial({'isolation_max_samples': min(256, n_samples) / n_samples, 'isolation_n_estimators': 100, 'proxy_n_estimators': 100})
    # Try (enqueue) some specific settings that led to good results during experiments
    study.enqueue_trial({'isolation_max_samples': 0.5492229999946834, 'isolation_n_estimators': 162, 'proxy_n_estimators': 153, 'proxy_max_depth': 10})
    study.enqueue_trial({'isolation_max_samples': 0.29110519961044856, 'isolation_n_estimators': 136, 'proxy_n_estimators': 77, 'proxy_max_depth': 8})
    
    study.enqueue_trial({'isolation_max_samples': 0.11350593116659227, 'isolation_n_estimators': 215, 'proxy_n_estimators': 112, 'proxy_max_depth': 15})
    study.enqueue_trial({'isolation_max_samples': 0.2646104863448817, 'isolation_n_estimators': 185, 'proxy_n_estimators': 109, 'proxy_max_depth': 8})

    study.optimize(objective, n_trials=number_of_trials, timeout=optimization_timeout_in_seconds)
    output_optuna_tuning_results(study, study.study_name)

    # Train best models from best params
    best_params = study.best_params

    best_isolation_forest = IsolationForest(
        n_estimators=best_params["isolation_n_estimators"],
        max_samples=best_params["isolation_max_samples"],
        contamination=contamination,
        random_state=random_seed
    )
    anomaly_detection_results = best_isolation_forest.fit_predict(feature_matrix)
    anomaly_labels = (anomaly_detection_results == -1).astype(int) # 1 = anomaly, 0 = normal
    anomaly_scores = best_isolation_forest.decision_function(feature_matrix) * -1  # higher = more anomalous

    best_proxy_random_forest = RandomForestClassifier(
        n_estimators=best_params["proxy_n_estimators"],
        max_depth=best_params["proxy_max_depth"],
        random_state=random_seed
    )
    best_proxy_random_forest.fit(feature_matrix, anomaly_detection_results)

    return AnomalyDetectionResults(anomaly_labels, anomaly_scores, best_proxy_random_forest, best_proxy_random_forest.feature_importances_)

In [ ]:
def add_anomaly_detection_results_to_features(
    features: pd.DataFrame,
    anomaly_detection_results: AnomalyDetectionResults,
    anomaly_label_column: str = 'anomalyLabel',
    anomaly_score_column: str = 'anomalyScore',
) -> pd.DataFrame:
    """
    Adds anomaly detection results to the feature and returns the updated dataframe.

    Parameters:
    - features: pd.DataFrame of shape (n_samples, n_features).
    - anomaly_detection_results: AnomalyDetectionResults object containing labels and scores.
    - anomaly_label_column: Name for the anomaly label column.
    - anomaly_score_column: Name for the anomaly score column.

    Returns:
    - Updated feature dataframe with anomaly labels and scores.
    """

    # Add anomaly labels and scores to the feature matrix
    features[anomaly_label_column] = anomaly_detection_results.anomaly_labels
    features[anomaly_score_column] = anomaly_detection_results.anomaly_scores
    return features

In [ ]:
def get_top_10_anomalies(
        anomaly_detected_features: pd.DataFrame, 
        anomaly_label_column: str = "anomalyLabel",
        anomaly_score_column: str = "anomalyScore"
) -> pd.DataFrame:
    anomalies = anomaly_detected_features[anomaly_detected_features[anomaly_label_column] == 1]
    return anomalies.sort_values(by=anomaly_score_column, ascending=False).reset_index(drop=True).head(10)

In [ ]:
java_package_anomaly_detection_results = tune_anomaly_detection_models(java_package_anomaly_detection_features_prepared)
java_package_anomaly_detection_features = add_anomaly_detection_results_to_features(java_package_anomaly_detection_features, java_package_anomaly_detection_results)
display(get_top_10_anomalies(java_package_anomaly_detection_features))

#### 1.3.b List the top 10 anomalies solely based on embeddings

In [ ]:
java_package_embedding_anomaly_detection_features = java_package_anomaly_detection_features[features_for_visualization_excluded_from_training + ['embedding', 'pageRank']].copy()
java_package_embedding_anomaly_detection_input = reduce_dimensionality_of_node_embeddings(java_package_embedding_anomaly_detection_features, max_dimensions=60, target_variance=0.95)
java_package_embedding_anomaly_detection_feature_names = embedding_feature_names = [f'pca_{i}' for i in range(java_package_embedding_anomaly_detection_input.shape[1])]
java_package_embedding_anomaly_detection_result = tune_anomaly_detection_models(java_package_embedding_anomaly_detection_input, contamination="auto")
java_package_embedding_anomaly_detection_features = add_anomaly_detection_results_to_features(java_package_embedding_anomaly_detection_features, java_package_embedding_anomaly_detection_result, anomaly_label_column='anomalyOfEmbeddingLabel', anomaly_score_column='anomalyOfEmbeddingScore')

display(get_top_10_anomalies(java_package_embedding_anomaly_detection_features, anomaly_label_column='anomalyOfEmbeddingLabel', anomaly_score_column='anomalyOfEmbeddingScore'))

### 1.4 Plot anomalies

Plots clustered nodes and highlights anomalies.

In [ ]:
def plot_anomalies(
    clustering_visualization_dataframe: pd.DataFrame,
    title_prefix: str,
    code_unit_column: str = "shortCodeUnitName",
    cluster_label_column: str = "clusterLabel",
    cluster_medoid_column: str = "clusterMedoid",
    cluster_size_column: str = "clusterSize",
    anomaly_label_column: str = "anomalyLabel",
    anomaly_score_column: str = "anomalyScore",
    page_rank_column: str = "pageRank",
    x_position_column: str = 'embeddingVisualizationX',
    y_position_column: str = 'embeddingVisualizationY',
) -> None:
    
    if clustering_visualization_dataframe.empty:
        print("No projected data to plot available")
        return
    
    def truncate(text: str, max_length: int):
        if len(text) <= max_length:
            return text
        return text[:max_length - 3] + "..."
    
    cluster_anomalies = clustering_visualization_dataframe[clustering_visualization_dataframe[anomaly_label_column] == 1]
    cluster_without_anomalies = clustering_visualization_dataframe[clustering_visualization_dataframe[anomaly_label_column] != 1]
    cluster_noise = cluster_without_anomalies[cluster_without_anomalies[cluster_label_column] == -1]
    cluster_non_noise = cluster_without_anomalies[cluster_without_anomalies[cluster_label_column] != -1]

    plot.figure(figsize=(10, 10))
    plot.title(title_prefix + ' (size=PageRank, color=ClusterLabel, red=Anomaly)')

    # Plot noise
    plot.scatter(
        x=cluster_noise[x_position_column],
        y=cluster_noise[y_position_column],
        s=cluster_noise[page_rank_column] * 200 + 4,
        color='lightgrey',
        alpha=0.5,
        label='Noise'
    )

    # Plot clusters
    plot.scatter(
        x=cluster_non_noise[x_position_column],
        y=cluster_non_noise[y_position_column],
        s=cluster_non_noise[page_rank_column] * 200 + 4,
        c=cluster_non_noise[cluster_label_column],
        cmap='tab20',
        alpha=0.7,
        label='Clusters'
    )

    # Plot anomalies
    plot.scatter(
        x=cluster_anomalies[x_position_column],
        y=cluster_anomalies[y_position_column],
        s=cluster_anomalies[page_rank_column] * 200 + 4,
        c=cluster_anomalies[anomaly_score_column],
        cmap="Reds",
        alpha=0.9,
        label='Anomaly'
    )

    # Annotate medoids of the cluster
    cluster_medoids = cluster_non_noise[cluster_non_noise[cluster_medoid_column] == 1].sort_values(by=cluster_size_column, ascending=False).head(20)
    for index, row in cluster_medoids.iterrows():
        plot.annotate(
            text=f"{row[cluster_label_column]}:{truncate(row[code_unit_column], 20)} ({row[anomaly_score_column]:.4f})",
            xy=(row[x_position_column], row[y_position_column]),
            xytext=(5, 5),
            alpha=0.4,
            **plot_annotation_style
        )

    anomalies = cluster_anomalies.sort_values(by=anomaly_score_column, ascending=False).reset_index(drop=True).head(10)
    for dataframe_index, row in anomalies.iterrows():
        index = typing.cast(int, dataframe_index)
        plot.annotate(
            text=f"{row[cluster_label_column]}:{truncate(row[code_unit_column], 20)} ({row[anomaly_score_column]:.4f})",
            xy=(row[x_position_column], row[y_position_column]),
            xytext=(5, 5 + (index % 5) * 10),
            color='red',
            **plot_annotation_style
        )

    plot.show()

In [ ]:
plot_anomalies(java_package_anomaly_detection_features, title_prefix="Java Package Anomalies")

#### 1.4.b Plot anomalies solely based on embeddings

In [ ]:
plot_anomalies(
    java_package_embedding_anomaly_detection_features, 
    title_prefix="Java Package Anomalies (Embeddings Only)",
    anomaly_label_column='anomalyOfEmbeddingLabel',
    anomaly_score_column='anomalyOfEmbeddingScore'
)

### 1.5 Print the 20 most influential features (no explainable AI yet)

Use Random Forest as a proxy to estimate the importance of each feature contributing to the anomaly score.

In [ ]:
java_package_anomaly_detection_importances_series = pd.Series(java_package_anomaly_detection_results.feature_importances, index=java_package_anomaly_detection_feature_names).sort_values(ascending=False)
print(java_package_anomaly_detection_importances_series.head(10))

### 1.6 Use SHAP to explain the Isolation Forest Model

In [ ]:
def explain_anomalies_with_shap(
        random_forest_model: RandomForestClassifier,
        prepared_features: np.ndarray
) -> np.ndarray:
    """
    Use SHAP to explain the detected anomalies.
    """
    # Use TreeExplainer on Random Forest Proxy Model
    # This is necessary because Isolation Forest does not support SHAP explanations directly.
    explainer = shap.TreeExplainer(random_forest_model)
    shap_values = explainer.shap_values(prepared_features)
    print(f"Input data with prepared features: shape={prepared_features.shape}")
    print(f"Explainable AI SHAP result: shape={np.shape(shap_values)}")
    return shap_values


In [ ]:
def plot_shap_explained_global_feature_importance(
    shap_values: numpy_typing.NDArray,
    anomaly_detected_features: pd.DataFrame,
    prepared_features: numpy_typing.NDArray,
    feature_names: list[str],
    title_prefix: str = "",
    anomaly_label_column: str = "anomalyLabel",
) -> None:
    """
    Explain anomalies using SHAP values and plot the global feature importance.
    This function uses the SHAP library to visualize the impact of features on the model's predictions
    for anomalies detected by the Isolation Forest model.
    It generates a bar plot showing the most influential features for the anomalies.
    """

    anomaly_rows = anomaly_detected_features[anomaly_label_column] == 1 # Filter anomalies
    shap.summary_plot(
        shap_values[anomaly_rows, :, 1],  # Class 1 = anomaly
        prepared_features[anomaly_rows],
        feature_names=feature_names,
        plot_type="bar",
        title=f"{title_prefix} Anomalies explained using SHAP",
        max_display=20,
        plot_size=(12, 6)  # (width, height) in inches
    )

In [ ]:
java_package_shap_values = explain_anomalies_with_shap(
    # random_forest_model=java_package_proxy_random_forest,
    random_forest_model=java_package_anomaly_detection_results.random_forest_classifier,
    prepared_features=java_package_anomaly_detection_features_prepared
)
plot_shap_explained_global_feature_importance(
    shap_values=java_package_shap_values,
    anomaly_detected_features=java_package_anomaly_detection_features, 
    prepared_features=java_package_anomaly_detection_features_prepared,
    feature_names=java_package_anomaly_detection_feature_names,
    title_prefix="Java Package"
)

### 1.7 Add the top SHAP features and their scores to the data frame

In [ ]:
def add_top_shap_features_to_anomalies(
    shap_values_array: np.ndarray,
    feature_names: list[str],
    anomaly_detected_features: pd.DataFrame,
    anomaly_label_column: str = "anomalyLabel",
    top_n: int = 3
) -> pd.DataFrame:
    """
    Adds top-N SHAP features and their SHAP values for each anomaly in the dataset.

    Parameters:
    - shap_values_array: SHAP values array with shape (n_samples, n_features).
    - feature_names: List of names corresponding to the features.
    - anomaly_detected_features: Original DataFrame containing anomaly labels.
    - anomaly_label_column: Name of the column indicating anomalies (1 = anomaly).
    - top_n: Number of top influential features to extract.

    Returns:
    - DataFrame with additional columns:
      anomalyTopFeature_1, ..., topFeature_N
      anomalyTopFeatureSHAPValue_, ..., topFeatureValue_N
    """
    # Convert SHAP values to DataFrame for easier processing
    shap_dataframe = pd.DataFrame(shap_values_array, columns=feature_names)

    # Get indices of rows marked as anomalies
    anomaly_indices = anomaly_detected_features[anomaly_detected_features[anomaly_label_column] == 1].index

    # Initialize result columns
    for rank in range(1, top_n + 1):
        anomaly_detected_features[f"anomalyTopFeature_{rank}"] = ""
        anomaly_detected_features[f"anomalyTopFeatureSHAPValue_{rank}"] = 0.0

    # Loop over each anomaly and assign top features + SHAP values
    for index in anomaly_indices:
        row_values = shap_dataframe.loc[index]
        top_features = row_values.abs().sort_values(ascending=False).head(top_n)
        for rank, (feature_name, shap_value) in enumerate(row_values[top_features.index].items(), 1):
            anomaly_detected_features.at[index, f"anomalyTopFeature_{rank}"] = feature_name
            anomaly_detected_features.at[index, f"anomalyTopFeatureSHAPValue_{rank}"] = shap_value

    return anomaly_detected_features


In [ ]:
add_top_shap_features_to_anomalies(
    shap_values_array=java_package_shap_values[:, :, 1],
    feature_names=java_package_anomaly_detection_feature_names,
    anomaly_detected_features=java_package_anomaly_detection_features
)
display(java_package_anomaly_detection_features[java_package_anomaly_detection_features["anomalyLabel"] == 1].sort_values(by='anomalyScore', ascending=False).head(10))

## 2. Java Types

### 2.1 Query Features

Query all features that are relevant for anomaly detection. Some of them come from precalculated clustering (HDBSCAN), node embeddings (Fast Random Projection), community detection algorithms (Leiden, Local Clustering Coefficient), centrality algorithms (Page Rank, Article Rank, Betweenness) and classical metrics like the in-/out-degree.


In [ ]:
java_type_anomaly_detection_features_query = """
    MATCH (artifact:Java:Artifact)-[:CONTAINS]->(codeUnit:Java:Type)
    WHERE codeUnit.incomingDependencies                              IS NOT NULL
      AND codeUnit.outgoingDependencies                              IS NOT NULL
      and codeUnit.embeddingsFastRandomProjectionTunedForClustering  IS NOT NULL
      AND codeUnit.centralityPageRank                                IS NOT NULL
      AND codeUnit.centralityArticleRank                             IS NOT NULL
      AND codeUnit.centralityBetweenness                             IS NOT NULL
      AND codeUnit.communityLocalClusteringCoefficient               IS NOT NULL
      AND codeUnit.clusteringHDBSCANProbability                      IS NOT NULL
      AND codeUnit.clusteringHDBSCANNoise                            IS NOT NULL
      AND codeUnit.clusteringHDBSCANMedoid                           IS NOT NULL
      AND codeUnit.clusteringHDBSCANRadiusAverage                    IS NOT NULL
      AND codeUnit.clusteringHDBSCANNormalizedDistanceToMedoid       IS NOT NULL
      AND codeUnit.clusteringHDBSCANLabel                            IS NOT NULL
      AND codeUnit.clusteringHDBSCANSize                             IS NOT NULL
      AND codeUnit.clusteringHDBSCANMedoid                           IS NOT NULL
      AND codeUnit.embeddingFastRandomProjectionVisualizationX       IS NOT NULL
      AND codeUnit.embeddingFastRandomProjectionVisualizationY       IS NOT NULL
   RETURN DISTINCT 
         codeUnit.fqn                                                  AS codeUnitName
        ,codeUnit.name                                                 AS shortCodeUnitName
        ,artifact.name                                                 AS projectName
        ,codeUnit.incomingDependencies                                 AS incomingDependencies
        ,codeUnit.outgoingDependencies                                 AS outgoingDependencies
        ,codeUnit.incomingDependencies + codeUnit.outgoingDependencies AS degree
        ,codeUnit.embeddingsFastRandomProjectionTunedForClustering     AS embedding
        ,codeUnit.centralityPageRank                                   AS pageRank
        ,codeUnit.centralityArticleRank                                AS articleRank
        ,codeUnit.centralityPageRank - codeUnit.centralityArticleRank  AS pageToArticleRankDifference
        ,codeUnit.centralityBetweenness                                AS betweenness
        ,codeUnit.communityLocalClusteringCoefficient                  AS locallusteringCoefficient
        ,1.0 - codeUnit.clusteringHDBSCANProbability                   AS clusterApproximateOutlierScore
        ,codeUnit.clusteringHDBSCANNoise                               AS clusterNoise
        ,codeUnit.clusteringHDBSCANRadiusAverage                       AS clusterRadiusAverage
        ,codeUnit.clusteringHDBSCANNormalizedDistanceToMedoid          AS clusterDistanceToMedoid
        ,codeUnit.clusteringHDBSCANLabel                               AS clusterLabel
        ,codeUnit.clusteringHDBSCANSize                                AS clusterSize
        ,codeUnit.clusteringHDBSCANMedoid                              AS clusterMedoid
        ,codeUnit.embeddingFastRandomProjectionVisualizationX          AS embeddingVisualizationX
        ,codeUnit.embeddingFastRandomProjectionVisualizationY          AS embeddingVisualizationY
"""

java_type_anomaly_detection_features = query_cypher_to_data_frame(java_type_anomaly_detection_features_query)
java_type_features_to_standardize = java_type_anomaly_detection_features.columns.drop(features_for_visualization_excluded_from_training + ['embedding']).to_list()

display(java_type_anomaly_detection_features.head(5))

### 1.2 Data preparation

Prepare the data by standardizing numeric fields and reducing the dimensionality of the node embeddings to not dominate the results.

In [ ]:
validate_data(java_type_anomaly_detection_features)
java_type_anomaly_detection_features_standardized = standardize_features(java_type_anomaly_detection_features, java_type_features_to_standardize)
java_type_anomaly_detection_node_embeddings_reduced = reduce_dimensionality_of_node_embeddings(java_type_anomaly_detection_features)

java_type_anomaly_detection_features_prepared = np.hstack([java_type_anomaly_detection_features_standardized, java_type_anomaly_detection_node_embeddings_reduced])
java_type_anomaly_detection_feature_names = list(java_type_features_to_standardize) + [f'pca_{i}' for i in range(java_type_anomaly_detection_node_embeddings_reduced.shape[1])]

plot_feature_correlation_matrix(java_type_anomaly_detection_features[java_type_features_to_standardize])

### 2.3 List the top 10 anomalies found using Isolation Forest

> The IsolationForest 'isolates' observations by randomly selecting a feature and then randomly selecting a split value between the maximum and minimum values of the selected feature.

In [ ]:
java_type_anomaly_detection_results = tune_anomaly_detection_models(java_type_anomaly_detection_features_prepared)
java_type_anomaly_detection_features = add_anomaly_detection_results_to_features(java_type_anomaly_detection_features, java_type_anomaly_detection_results)
display(get_top_10_anomalies(java_type_anomaly_detection_features))

#### 2.3.b List the top 10 anomalies solely based on embeddings

In [ ]:
java_type_embedding_anomaly_detection_features = java_type_anomaly_detection_features[features_for_visualization_excluded_from_training + ['embedding', 'pageRank']].copy()
java_type_embedding_anomaly_detection_input = reduce_dimensionality_of_node_embeddings(java_type_embedding_anomaly_detection_features, max_dimensions=60, target_variance=0.95)
java_type_embedding_anomaly_detection_feature_names = embedding_feature_names = [f'pca_{i}' for i in range(java_type_embedding_anomaly_detection_input.shape[1])]
java_type_embedding_anomaly_detection_result = tune_anomaly_detection_models(java_type_embedding_anomaly_detection_input, contamination="auto")
java_type_embedding_anomaly_detection_features = add_anomaly_detection_results_to_features(java_type_embedding_anomaly_detection_features, java_type_embedding_anomaly_detection_result, anomaly_label_column='anomalyOfEmbeddingLabel', anomaly_score_column='anomalyOfEmbeddingScore')

display(get_top_10_anomalies(java_type_embedding_anomaly_detection_features, anomaly_label_column='anomalyOfEmbeddingLabel', anomaly_score_column='anomalyOfEmbeddingScore'))

### 2.4. Plot anomalies

Plots clustered nodes and highlights anomalies.

In [ ]:
plot_anomalies(java_type_anomaly_detection_features, title_prefix="Java Type Anomalies")

#### 2.4.b Plot anomalies solely based on embeddings

In [ ]:
plot_anomalies(
    java_type_embedding_anomaly_detection_features, 
    title_prefix="Java Type Anomalies (Embeddings Only)",
    anomaly_label_column='anomalyOfEmbeddingLabel',
    anomaly_score_column='anomalyOfEmbeddingScore'
)

### 2.5 Print the 20 most influential features (no explainable AI yet)

Use Random Forest as a proxy to estimate the importance of each feature contributing to the anomaly score.

In [ ]:
java_type_anomaly_detection_importances_series = pd.Series(java_type_anomaly_detection_results.feature_importances, index=java_type_anomaly_detection_feature_names).sort_values(ascending=False)
print(java_type_anomaly_detection_importances_series.head(10))

### 2.6 Use SHAP to explain the Isolation Forest Model via the Random Forest Proxy

In [ ]:
java_type_shap_values = explain_anomalies_with_shap(
    random_forest_model=java_type_anomaly_detection_results.random_forest_classifier,
    prepared_features=java_type_anomaly_detection_features_prepared
)
plot_shap_explained_global_feature_importance(
    shap_values=java_type_shap_values,
    anomaly_detected_features=java_type_anomaly_detection_features, 
    prepared_features=java_type_anomaly_detection_features_prepared,
    feature_names=java_type_anomaly_detection_feature_names,
    title_prefix="Java Package"
)

### 2.7 Add the top SHAP features and their scores to the data frame

In [ ]:
add_top_shap_features_to_anomalies(
    shap_values_array=java_type_shap_values[:, :, 1],
    feature_names=java_type_anomaly_detection_feature_names,
    anomaly_detected_features=java_type_anomaly_detection_features
)
display(java_type_anomaly_detection_features[java_type_anomaly_detection_features["anomalyLabel"] == 1].sort_values(by='anomalyScore', ascending=False).head(10))